# **요약 텍스트 형식 변환**

## process train dataset

In [ ]:
import os
import json
import re

# 폴더 경로 설정
folder_path = './train_2~3sent'  
json_files = [file for file in os.listdir(folder_path) if file.endswith('.json')]

data_items = []

# 각 파일의 내용을 순차적으로 읽음
for json_file in json_files:
    file_path = os.path.join(folder_path, json_file)
    
    with open(file_path, 'r', encoding='utf-8') as file:
        train_json_dict = json.load(file)
        sentences = re.split(r'(?<=[.?!]) +|\n|\\n', train_json_dict["Meta(Refine)"]['passage'])
        child = [value for value in train_json_dict["Annotation"].values() if value is not None]



        # child의 각 요소가 sentences 리스트의 어느 인덱스에 해당하는지 찾기
        indexes = []
        for c in child:
            if c in sentences:
                indexes.append(sentences.index(c))
        print(indexes)
        data = {'id':train_json_dict["Meta(Acqusition)"]['doc_id'],"article_original":sentences,"extractive_sents": [value for value in train_json_dict["Annotation"].values() if value is not None]}
        data_items.append(data)

In [23]:
child = [value for value in train_json_dict["Annotation"].values() if value is not None]

# child의 각 요소가 sentences 리스트의 어느 인덱스에 해당하는지 찾기
indexes = []
for c in child:
    if c in sentences:
        indexes.append(sentences.index(c))


[]

In [21]:
# JSON Lines 파일로 저장
with open('train_2.jsonl', 'w', encoding='utf-8') as file:
    for item in data_items:
        json_line = json.dumps(item, ensure_ascii=False) + '\n'  # 각 항목을 JSON 문자열로 변환하고 줄바꿈 추가
        file.write(json_line)

print("Data saved to 'data.jsonl'.")

Data saved to 'data.jsonl'.


## process valid dataset

In [10]:
import os
import json

# 폴더 경로 설정
folder_path = './valid_2~3sent'  
json_files = [file for file in os.listdir(folder_path) if file.endswith('.json')]

data_items = []

# 각 파일의 내용을 순차적으로 읽음
for json_file in json_files:
    file_path = os.path.join(folder_path, json_file)
    
    with open(file_path, 'r', encoding='utf-8') as file:
        train_json_dict = json.load(file)
        data = {'id':train_json_dict["Meta(Acqusition)"]['doc_id'],"article_original":train_json_dict["Meta(Refine)"]['passage'],"extractive_sents": [value for value in train_json_dict["Annotation"].values() if value is not None]}
        data_items.append(data)

In [12]:
# JSON Lines 파일로 저장
with open('valid_2.jsonl', 'w', encoding='utf-8') as file:
    for item in data_items:
        json_line = json.dumps(item, ensure_ascii=False) + '\n'  # 각 항목을 JSON 문자열로 변환하고 줄바꿈 추가
        file.write(json_line)

print("Data saved to 'data.jsonl'.")

Data saved to 'data.jsonl'.


# **데이터프레임 데이터셋 구축**

In [99]:
import pandas as pd
data = pd.read_csv('./initial_data_final.csv')

In [133]:
import re
def add_space_after_period(text):
    # 마침표 뒤에 숫자가 아닌 문자가 오면 그 사이에 공백을 추가
    text1 = re.sub(r'\.(?![0-9])', '. ', text)
    text2 = re.sub(r'\?(?![0-9])', '? ', text1)
    modified_text = re.sub(r'\!(?![0-9])', '! ', text2)
    return modified_text
def split_sentences_with_decimal(text):
    # 숫자가 아닌 문자 뒤에 오는 마침표를 기준으로 분리
    # 숫자 다음에 오는 마침표는 분리하지 않음
    sentences = re.split(r'(?<!\d)\.(?=\s|$|\n)', text)
    # 분리된 각 부분의 끝에 마침표를 추가
    sentences = [sentence.strip() + '.' for sentence in sentences if sentence.strip()]
    return sentences


In [132]:
import json
data_items = []
import re

with open('test.jsonl', 'w', encoding='utf-8') as file:
    idx=1
    for index, row in data.iterrows():
        body = str(row['Body'])

        formatted_text = add_space_after_period(row['Body'])
        sentences = re.split(r'(?<=[.?!]) +|\n|\\n', formatted_text)
        # 불필요한 요소 제거 ( '.', 'r', 빈 문자열)
        sentences = [s for s in sentences if s not in {'.', ' ','\r',' \r' ,'','\"\r'}]
   
        data_dict = {'id': idx, 'article_original': sentences}
        json_line = json.dumps(data_dict, ensure_ascii=False) + '\n'  # 각 항목을 JSON 문자열로 변환하고 줄바꿈 추가
        file.write(json_line)
        idx+=1
        
 

## 요약문으로 대체

In [135]:
import pandas as pd
sum_data = pd.read_csv('C:/Users/hws07/Desktop/python_folder/KoBertSum/ext/data/raw/extractive_sample_submission_body.csv')
origin_data = pd.read_csv('./initial_data_final.csv')

In [139]:
origin_data['Body'] = sum_data['summary']

In [141]:
origin_data.to_csv('./initial_data_final_v2.csv',encoding='utf-8')